## Cтроим базовый RAG

![structure.png](/Users/anastasia/Desktop/ML/Диплом/Schemes/Base_RAG.png)

### Библиотеки и environ

In [31]:
from necessity import *

# LangSmith
os.environ["LANGCHAIN_PROJECT"] = "BASE_RAG"

### Загружаем документы

In [ ]:
# загрузка документов
loader = PyPDFLoader("../data/Пользовательское соглашение Тинькофф")
docs = loader.load()
# разбивка документов
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000)
splits = text_splitter.split_documents(docs)

llm = GigaChat(verify_ssl_certs=False, scope="GIGACHAT_API_PERS")
embedding_model  = GigaChatEmbeddings(scope="GIGACHAT_API_PERS", verify_ssl_certs=False)
db = FAISS.from_documents(documents=splits, embedding=embedding_model)

# устанавливаем ретриевер для поиска контекста
retriever = db.as_retriever(
    search_type="mmr",
    search_kwargs={'k': 1, 'lambda_mult': 0.25}
)

# задаем промт
prompt = ChatPromptTemplate.from_template("""Вы являетесь помощником в выполнении поиска ответов на вопросы.
                                          Используйте приведенные ниже фрагменты извлеченного контекста, чтобы ответить на вопрос. 
                                          Если вы не знаете ответа, просто скажите, что вы не знаете.
        Question: {question} 
        Context: {context} 
        Answer:""")

def format_docs(docs):
    for elem in docs:
        print(elem.page_content)
    print('_____________')
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

answer = rag_chain.invoke("Расскажи про термины 'Договор страхования' и 'КАСКО")

In [33]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("../data/Пользовательское соглашение Тинькофф/3_Пользовательское соглашение АО «Тинькофф Страхование».pdf")
docs = loader.load()

Ignoring wrong pointing object 6 0 (offset 0)
Ignoring wrong pointing object 8 0 (offset 0)
Ignoring wrong pointing object 10 0 (offset 0)
Ignoring wrong pointing object 22 0 (offset 0)
Ignoring wrong pointing object 24 0 (offset 0)
Ignoring wrong pointing object 26 0 (offset 0)
Ignoring wrong pointing object 190 0 (offset 0)


In [34]:
docs

[Document(page_content='1 из 6          Пользовательское соглашение      АО «Тинькофф Страхование» (далее — Соглашение)     г. Москва      06 октября 2023 года     Настоящее Соглашение является публичной офертой и определяет условия использования материалов и сервисов, размещенных на официальном сайте АО «Тинькофф Страхование» (далее — Страховщик) в сети Интернет по адресу: tinkoﬃnsurance.ru, посетителями и пользователями данного интернет-сайта (далее — Сайт), а также условия и порядок дистанционного взаимодействия со Страховщиком с использованием Сайта и/или сайтов в сети Интернет страховых агентов и страховых брокеров Страховщика, а также аффилированных лиц Страховщика и/или их мобильных приложений.     1. Термины и определения     Договор страхования — договор, по которому одна сторона (Страховщик) обязуется за обусловленную договором плату (страховую премию) при наступлении предусмотренного в договоре события (страхового случая) осуществить другой стороне (страхователю) или иному л

### Разбиваем документы на чанки

In [35]:
# text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000)
splits = text_splitter.split_documents(docs)

### Индексируем наши чанки 

In [37]:
from langchain_community.embeddings.gigachat import GigaChatEmbeddings

embedding_model  = GigaChatEmbeddings(scope="GIGACHAT_API_PERS", verify_ssl_certs=False)
db = FAISS.from_documents(documents=splits, embedding=embedding_model)

# устанавливаем ретриевер для поиска контекста
retriever = db.as_retriever(
    search_type="mmr",
    search_kwargs={'k': 1, 'lambda_mult': 0.25}
)

### Загружаем LLM

In [42]:
from langchain_community.llms import GigaChat

llm = GigaChat(verify_ssl_certs=False, scope="GIGACHAT_API_PERS")

### Строим prompt-инструкцию для LLM

In [43]:
prompt = ChatPromptTemplate.from_template("""Вы являетесь помощником в выполнении поиска ответов на вопросы. Используйте приведенные ниже фрагменты извлеченного контекста, чтобы ответить на вопрос. Если вы не знаете ответа, просто скажите, что вы не знаете.
        Question: {question} 
        Context: {context} 
        Answer:""")

### Собираем RAG-CHAIN
` {"context": retriever | format_docs, "question": RunnablePassthrough()}`


`"context": retriever | format_docs` - контекст строится по цепочке 
1. `retriever` возвращает нам близкие чанки. 
2. `format_docs` форматирует текст чанков.

`"question": RunnablePassthrough()` - позволяет нам передать вопрос от пользователя в цепочку `retriever | format_docs` для получения контекста 


Таким обрзом при вызове  RAG цепочки
1. формируется словарь: `{"context": "context from retriver", "question": "user question"}`
2. словарь отпраляется в `prompt`.
3. `prompt` подается в LLM.
4.  LLM возвращает ответ и ` StrOutputParser()` просто отдает его назад без обработки. 

In [44]:
def format_docs(docs):
    for elem in docs:
        print(elem.page_content)
    print('_____________')
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

answer = rag_chain.invoke("Расскажи про термины 'Договор страхования' и 'КАСКО")

### Красивый вывод

In [15]:
from IPython.display import HTML

# Создаем HTML-ячейку
html_cell = HTML(f"<pre style='background-color: #efefef; padding: 10px; border: 1px solid #ccc; border-radius: 5px; white-space: pre-wrap; line-height: 1.5rem;'>{answer}</pre>")

display(html_cell)